# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/workspace/home
['/workspace/home/event_data/2018-11-17-events.csv', '/workspace/home/event_data/2018-11-18-events.csv', '/workspace/home/event_data/2018-11-14-events.csv', '/workspace/home/event_data/2018-11-02-events.csv', '/workspace/home/event_data/2018-11-16-events.csv', '/workspace/home/event_data/2018-11-22-events.csv', '/workspace/home/event_data/2018-11-24-events.csv', '/workspace/home/event_data/2018-11-11-events.csv', '/workspace/home/event_data/2018-11-12-events.csv', '/workspace/home/event_data/2018-11-15-events.csv', '/workspace/home/event_data/2018-11-29-events.csv', '/workspace/home/event_data/2018-11-04-events.csv', '/workspace/home/event_data/2018-11-25-events.csv', '/workspace/home/event_data/2018-11-03-events.csv', '/workspace/home/event_data/2018-11-21-events.csv', '/workspace/home/event_data/2018-11-26-events.csv', '/workspace/home/event_data/2018-11-06-events.csv', '/workspace/home/event_data/2018-11-19-events.csv', '/workspace/home/event_data/2018-11-01-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster

try: 
    cluster = Cluster(['127.0.0.1'])

    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [7]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS locnd15_project_1 
    WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('locnd15_project_1')
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [21]:
query = "DROP TABLE music_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)


In [22]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = "CREATE TABLE IF NOT EXISTS music_library "
query = query + "(artist text, song_title text, song_length float, session_id int, item_in_session int, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)


In [ ]:
file = 'event_datafile_new.csv'


with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    cnt = 0
    for line in csvreader:
        print(f"artist: {line[0]}\n item_in_session:{line[3]}\n length:{line[5]}\n session_id:{line[8]}\n song: {line[9]}\n\n")
        cnt += 1
        if cnt == 5: break
        

In [24]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_library (artist, song_title, song_length, session_id, item_in_session)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        
        artist = line[0].strip()
        item_in_session = int(line[3])
        length = float(line[5])
        session_id = int(line[8])
        song = line[9].strip()
        session.execute(query, (artist, song, length, session_id, item_in_session))

#### Do a SELECT to verify that the data have been inserted into each table

In [27]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = "SELECT artist, song_title, song_length FROM music_library WHERE session_id=338 AND item_in_session=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (f"{row.artist}, {row.song_title}, {row.song_length}")

Faithless, Music Matters (Mark Knight Dub), 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [51]:
query = "DROP TABLE artist_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)


In [52]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query = """CREATE TABLE IF NOT EXISTS artist_library
(artist text, song_title text, first_name text, last_name text, item_in_session int, user_id int, session_id int,
PRIMARY KEY ((user_id, session_id), item_in_session))
WITH CLUSTERING ORDER BY (item_in_session ASC)"""
try:
    session.execute(query)
except Exception as e:
    print(e)


In [53]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_library (artist, song_title, first_name, last_name, item_in_session, user_id, session_id)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        artist = line[0].strip()
        first_name = line[1].strip()
        last_name = line[4].strip()
        song = line[9].strip()
        item_in_session = int(line[3])
        user_id = int(line[10])
        session_id = int(line[8])

        session.execute(query, (artist, song, first_name, last_name, item_in_session, user_id, session_id))

In [54]:
query = "SELECT item_in_session, artist, song_title, first_name, last_name FROM artist_library WHERE user_id=10 AND session_id=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (f"{row.item_in_session} | {row.artist} | {row.song_title} | {row.first_name} {row.last_name}")
    

0 | Down To The Bone | Keep On Keepin' On | Sylvie Cruz
1 | Three Drives | Greece 2000 | Sylvie Cruz
2 | Sebastien Tellier | Kilometer | Sylvie Cruz
3 | Lonnie Gordon | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) | Sylvie Cruz


In [62]:
query = "DROP TABLE user_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)


In [63]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query = """CREATE TABLE IF NOT EXISTS user_library
(song_title text, first_name text, last_name text, user_id int,
PRIMARY KEY (song_title, user_id))"""

# query = """CREATE TABLE IF NOT EXISTS user_library
# (song_title text, first_name text, last_name text,
# PRIMARY KEY (song_title))"""
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [65]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_library (song_title, first_name, last_name, user_id)"
        query = query + " VALUES (%s, %s, %s, %s)"
        
        song = line[9].strip()
        first_name = line[1].strip()
        last_name = line[4].strip()
        user_id = int(line[10])
        
        session.execute(query, (song, first_name, last_name, user_id))

In [66]:
query = "SELECT first_name, last_name FROM user_library WHERE song_title='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (f"{row.first_name} {row.last_name}")
    

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [4]:
## TO-DO: Drop the table before closing out the sessions

In [ ]:
query = "DROP TABLE IF EXISTS music_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS artist_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS user_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()